In [57]:
# create a model that uses the fenchel-young loss and trains on 200 images 
# the images are permuted in 9 tiles and in every iteration a new permutation is generated.
# we use a deep encoder with fixed parameters, adam optimizer, preprocessing before tiling  

import tensorflow as tf

In [58]:
from ast import Pass
import platform
import cv2
import numpy as np
import os
from pathlib import Path

number_of_images = 250

root = os.getcwd() # Don't change this
data_dirname = '/data_test/plantvillage/' # Change as you like 
p = Path(root + data_dirname)
p.mkdir(exist_ok=True) 
if platform.system()=='Darwin':
  root = os.getcwd() # Don't change this
  data_dirname = '/data_test/plantvillage/' # Change as you like 
  p = Path(root + data_dirname)
  p.mkdir(exist_ok=True) 
else:
  #p = Path("C:/Users/mwels/Documents/Uni/11. Semester/Deep learning in visual recognition/Plant_leave_diseases_dataset_without_augmentation")
  #p.mkdir(exist_ok=True)
  pass



classes = [
  'Apple___Apple_scab',
  'Apple___healthy',
  'Apple___Black_rot',
  'Apple___Cedar_apple_rust',
  "all"
  ]

if "all" in classes:
  classes = os.listdir(p)

for c in classes:
  print(c,end=" ")
  filelist = [x for x in (p/c).iterdir() if x.is_file()]
  for f in filelist:
    img = cv2.imread(str(f))
    if img is None:
      print(f'Failed to open {f}. Deleting file')
      os.remove(str(f))


filelist = filelist[:number_of_images]
print(len(filelist))

Apple___Apple_scab Apple___Black_rot Apple___Cedar_apple_rust Apple___healthy 250


In [59]:
len(filelist)

250

In [60]:
tilex = 3
number_of_tiles = tilex
tile_size = 255//tilex

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from keras.applications.mobilenet import MobileNet
from keras.applications.mobilenet import preprocess_input

conv_base = MobileNet(weights='imagenet',
                      include_top=False,
                      input_shape=(tile_size, tile_size, 3))

In [61]:
sys.path.append('src')

In [62]:
from keras.layers import Dense, Concatenate, Input, Flatten, Lambda, BatchNormalization
from keras.models import Model
from model_tools import Ranking
import perturbations
import os
import keras.backend as K

tiles = Input((number_of_tiles,tile_size,tile_size,3))
layers = {}
embedds = {}

shared_conv = conv_base 

for i in range(number_of_tiles):
    layers[f'tile{i}'] = Lambda(lambda x: x[:,i,:,:,:])(tiles)

    layers[f'deep_layers{i}'] = shared_conv(layers[f'tile{i}'])
    embedds[f'embedd{i}'] = Flatten()(layers[f'deep_layers{i}'])

concatonation = Concatenate(axis=1)(list(embedds.values()))

out = Dense(number_of_tiles*10, activation="relu", kernel_initializer='he_normal')(concatonation)
out = BatchNormalization()(out)
out = Dense(number_of_tiles, kernel_initializer='he_normal')(out)
out = Flatten()(out)
out = Ranking()(out) 

model = Model(inputs=tiles, outputs=out)

In [63]:
tf.keras.utils.plot_model(model, 
    show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [64]:
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_10 (InputLayer)          [(None, 3, 85, 85,   0           []                               
                                3)]                                                               
                                                                                                  
 lambda_10 (Lambda)             (None, 85, 85, 3)    0           ['input_10[0][0]']               
                                                                                                  
 lambda_11 (Lambda)             (None, 85, 85, 3)    0           ['input_10[0][0]']               
                                                                                                  
 lambda_12 (Lambda)             (None, 85, 85, 3)    0           ['input_10[0][0]']         

In [65]:
total_num_layers = len(model.layers)
num_base_layers = len(conv_base.layers)
print(f"Total number of layers is {total_num_layers}")
print(f"Number of pretrained base layers is {num_base_layers}")

for layer in model.layers[:num_base_layers]:
    layer.trainable=False
for layer in model.layers[num_base_layers:]:
    layer.trainable=True

Total number of layers is 14
Number of pretrained base layers is 86


In [66]:
from keras import optimizers
from sklearn.model_selection import train_test_split
from src.PermSubsetDataGen import *
from src.model_tools import *
from src.permutation_tools import *

x_train, x_test = train_test_split(filelist)

train_generator_s = PermSubsetDataGen(x_train,
                                    batch_size=8,
                                    tilenumberx=tilex, vert=True)

validation_generator_s = PermSubsetDataGen(x_test,
                                        batch_size=8,
                                        tilenumberx=tilex, vert=True)


optimizer = tf.keras.optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    name='Adam',
)


model.compile(
    optimizer=optimizer,
#    loss=RankingLoss(),
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=[ProjectedRanksAccuracy(), PartialRanksAccuracy(), 'accuracy'])

In [67]:
print(train_generator_s.next()[1].shape)
print(validation_generator_s.next()[1].shape)

(8, 3)
(8, 3)


In [68]:
import numpy as np 
from tensorflow import keras
from matplotlib import pyplot as plt
from IPython.display import clear_output

class PlotLearning(tf.keras.callbacks.Callback):
    """
    Callback to plot the learning curves of the model during training.
    """
    def on_train_begin(self, logs={}):
        self.metrics = {}
        for metric in logs:
            self.metrics[metric] = []
            

    def on_epoch_end(self, epoch, logs={}):
        # Storing metrics
        for metric in logs:
            if metric in self.metrics:
                self.metrics[metric].append(logs.get(metric))
            else:
                self.metrics[metric] = [logs.get(metric)]
        
        # Plotting
        metrics = [x for x in logs if 'val' not in x]
        
        f, axs = plt.subplots(1, len(metrics), figsize=(15,5))
        clear_output(wait=True)

        for i, metric in enumerate(metrics):
            axs[i].plot(range(1, epoch + 2), 
                        self.metrics[metric], 
                        label=metric)
            if logs['val_' + metric]:
                axs[i].plot(range(1, epoch + 2), 
                            self.metrics['val_' + metric], 
                            label='val_' + metric)
                
            axs[i].legend()
            axs[i].grid()

        plt.tight_layout()
        plt.show()

In [69]:
nb_epochs = 30

model.fit(train_generator_s,
          epochs = nb_epochs,
          validation_data=validation_generator_s,
          verbose=1)

Epoch 1/30
24/24 [==============================] - 12s 366ms/step - loss: 1.3918 - projection_ranks_acc: 0.5760 - partial_ranks_acc: 0.3177 - accuracy: 0.2674 - val_loss: 1.0956 - val_projection_ranks_acc: 0.6638 - val_partial_ranks_acc: 0.4107 - val_accuracy: 0.4127
Epoch 2/30
24/24 [==============================] - 8s 322ms/step - loss: 1.3648 - projection_ranks_acc: 0.5788 - partial_ranks_acc: 0.2992 - accuracy: 0.3155 - val_loss: 1.0874 - val_projection_ranks_acc: 0.6670 - val_partial_ranks_acc: 0.4301 - val_accuracy: 0.4286
Epoch 3/30
24/24 [==============================] - 8s 322ms/step - loss: 1.3024 - projection_ranks_acc: 0.5913 - partial_ranks_acc: 0.3200 - accuracy: 0.3209 - val_loss: 1.2858 - val_projection_ranks_acc: 0.6080 - val_partial_ranks_acc: 0.3162 - val_accuracy: 0.3651
Epoch 4/30
24/24 [==============================] - 8s 323ms/step - loss: 1.2967 - projection_ranks_acc: 0.5983 - partial_ranks_acc: 0.3553 - accuracy: 0.3369 - val_loss: 1.3879 - val_projection_

KeyboardInterrupt: 

In [ ]:
#model.save("models/2022_10_27__01")

INFO:tensorflow:Assets written to: models/2022_10_27__01/assets


INFO:tensorflow:Assets written to: models/2022_10_27__01/assets
